In [2]:
import json
import string
import typing


In [3]:
def create_bank_from_sources() -> None:
    TARGET_SIZE = 5

    def filter_words(words: typing.Iterable[str]) -> typing.Iterable[str]:
        for word in words:
            if len(word) != TARGET_SIZE:
                continue
            if any(c not in string.ascii_lowercase for c in word):
                continue
            yield word

    def load_words_from_json(path: str) -> typing.Iterable[str]:
        return filter_words(json.load(open(path, 'r')))

    def load_words_from_txt(path: str) -> typing.Iterable[str]:
        lines = open(path, 'r').readlines()
        return filter_words(map(str.strip, lines))

    guesses = set(
        load_words_from_json('./bank_source/co_wordle_5_letter_guesses.json'))
    raw_targets = set(
        load_words_from_json('./bank_source/dordle_wordle_targets.json'))
    targets = raw_targets & guesses

    common_words = set(
        list(load_words_from_txt('./bank_source/cspell_en_gb_dict.txt')) +
        list(load_words_from_txt('./bank_source/cspell_en_us_dict.txt')))
    common_guesses = guesses & common_words

    def dump_words_to_txt(words: typing.Iterable[str], path: str) -> None:
        lines = sorted(w + '\n' for w in words)
        open(path, 'w').writelines(lines)

    dump_words_to_txt(guesses, './out_guesses.txt')
    dump_words_to_txt(common_guesses, './out_common_guesses.txt')
    dump_words_to_txt(targets, './out_targets.txt')

    print(f'{len(guesses)=} {len(common_guesses)=} {len(targets)=}')


create_bank_from_sources()


len(guesses)=12672 len(common_guesses)=5677 len(targets)=2391
